# 날씨, 토양도, 임상도, 산사태 발생 이력을 병합하는 코드
모든 데이터를 하나로 결합한 최종 데이터 생성

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks')

In [ ]:
# 필수 라이브러리

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
# 합칠 데이터들 불러오기

# ASOS 데이터 불러오기
asos = pd.read_csv('ASOS_umd.csv')
# 토양도 데이터 불러오기
soil = pd.read_csv("y_sum_hg_all.csv")
# 임상도 데이터 불러오기
tree = pd.read_csv('tree_sum_final.csv', encoding = 'cp949')

# 불필요한 변수 제거
soil = soil.drop(['umd', "sgg", "Unnamed: 0", "sum_cnt", "landslide"], axis=1)
asos = asos.drop(['umd', "sgg", "stnNm"], axis=1)
tree = tree.drop(['umd', "sgg"], axis=1)

In [ ]:
# 임상도, 토양도 병합
# 시와 읍면동 기준으로 생성한 umd2 기준으로 병합

merge_s_t = pd.merge(soil,tree, how='outer',on='umd2')
merge_s_t = merge_s_t.fillna(0)
merge_s_t

,umd2,PRRCK_LARG,PRRCK_MDDL,LOCTN_ALTT,LOCTN_GRDN,EIGHT_AGL,CLZN_CD,TPGRP_TPCD,PRDN_FOM_C,SLANT_TYP,SLDPT_TPCD,SCSTX_CD,SLTP_CD,STORUNST,FROR_CD,FRTP_CD,KOFTR_GROU,DMCLS_CD,AGCLS_CD,DNST_CD,HEIGHT
0,고령군 개진면,2,24,306.1,18.7,158.6,3,12,1,1,20,2,1,1.0,1.0,3.0,77.0,0.0,1.0,A,0.0
1,고령군 성산면,2,26,103.2,13.4,244.5,3,12,1,1,10,2,1,1.0,1.0,3.0,77.0,0.0,1.0,A,0.0
2,고령군 우곡면,2,24,64.1,17.7,285.2,3,7,1,1,10,3,13,1.0,2.0,1.0,11.0,2.0,4.0,C,14.0
3,청도군 각북면,1,12,573.4,22.5,15.5,3,12,1,1,10,2,1,1.0,2.0,1.0,11.0,2.0,4.0,C,14.0
4,청도군 이서면,2,26,249.5,15.1,86.6,3,12,1,1,10,2,1,1.0,2.0,2.0,30.0,2.0,4.0,C,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
632,진주시 신안동,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
633,진주시 충무공동,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,1.0,2.0,2.0,30.0,1.0,3.0,C,10.0
634,통영시 욕지면,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,1.0,2.0,2.0,30.0,2.0,4.0,C,10.0
635,통영시 사량면,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,1.0,2.0,2.0,30.0,2.0,4.0,C,14.0


In [ ]:
# 날씨 데이터와 토양도+임상도 데이터 병합

merge_s_h_a = pd.merge(asos, merge_s_t, how='outer',on='umd2')
merge_s_h_a

,tm,stnId,umd2,avgTa,minTa,maxTa,mi10MaxRn,hr1MaxRn,sumRnDur,sumRn,maxInsWs,maxWs,avgWs,minRhm,avgRhm,n99Rn,PRRCK_LARG,PRRCK_MDDL,LOCTN_ALTT,LOCTN_GRDN,EIGHT_AGL,CLZN_CD,TPGRP_TPCD,PRDN_FOM_C,SLANT_TYP,SLDPT_TPCD,SCSTX_CD,SLTP_CD,STORUNST,FROR_CD,FRTP_CD,KOFTR_GROU,DMCLS_CD,AGCLS_CD,DNST_CD,HEIGHT
0,2011-01-01,115,울릉군 울릉읍,0.8,-1.1,2.1,0.0,0.0,19.33,28.0,29.4,19.0,11.1,65.0,82.5,5.8,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,1.0,2.0,2.0,30.0,1.0,5.0,C,16.0
1,2011-01-02,115,울릉군 울릉읍,0.1,-0.9,1.3,0.0,0.0,22.83,24.8,14.5,8.5,6.0,65.0,91.5,29.7,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,1.0,2.0,2.0,30.0,1.0,5.0,C,16.0
2,2011-01-03,115,울릉군 울릉읍,-0.3,-1.4,0.8,0.0,0.0,12.83,7.5,15.9,10.2,6.5,61.0,80.3,0.3,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,1.0,2.0,2.0,30.0,1.0,5.0,C,16.0
3,2011-01-04,115,울릉군 울릉읍,1.7,0.4,3.9,0.0,0.0,0.00,0.0,11.2,6.8,3.1,53.0,63.1,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,1.0,2.0,2.0,30.0,1.0,5.0,C,16.0
4,2011-01-05,115,울릉군 울릉읍,1.2,-1.2,2.6,0.0,0.0,7.25,1.8,13.9,6.9,4.1,42.0,61.5,12.3,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,1.0,2.0,2.0,30.0,1.0,5.0,C,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2093785,2019-12-27,295,남해군 창선면,3.3,-1.9,7.4,0.0,0.0,0.00,0.0,5.8,3.4,1.8,30.0,50.0,0.0,1,11,204.6,31.4,322.5,4,7,1,1,20,2,1,1.0,2.0,1.0,15.0,2.0,4.0,C,12.0
2093786,2019-12-28,295,남해군 창선면,2.2,-1.9,8.1,0.0,0.0,0.00,0.0,2.7,1.6,0.9,33.0,56.1,0.0,1,11,204.6,31.4,322.5,4,7,1,1,20,2,1,1.0,2.0,1.0,15.0,2.0,4.0,C,12.0
2093787,2019-12-29,295,남해군 창선면,3.2,-0.5,5.8,0.0,0.0,0.00,8.5,2.6,1.4,0.8,53.0,75.8,0.0,1,11,204.6,31.4,322.5,4,7,1,1,20,2,1,1.0,2.0,1.0,15.0,2.0,4.0,C,12.0
2093788,2019-12-30,295,남해군 창선면,6.8,3.6,12.6,0.0,0.0,0.00,0.5,6.1,3.4,1.2,50.0,78.9,0.0,1,11,204.6,31.4,322.5,4,7,1,1,20,2,1,1.0,2.0,1.0,15.0,2.0,4.0,C,12.0


In [ ]:
# 병합한 데이터를 merge_s_h_a.csv 라는 이름으로 저장

# 불필요한 변수 제거
merge_s_h_a = merge_s_h_a.drop(["DNST_CD"], axis=1)
merge_s_h_a.to_csv("merge_s_h_a.csv", index=False)

# 날씨, 임상도, 토양도를 합친 데이터에 산사태 발생 이력 병합

In [ ]:
# 날짜형 데이터
merge_s_h_a["tm"] = pd.to_datetime(merge_s_h_a["tm"])

In [ ]:
# 새로운 산사태 발생 이력 데이터프레임 생성

# 산사태 발생 이력 데이터 불러오기
landslide_CNT = pd.read_csv('landslide_history.csv', encoding='cp949')

# 산사태 발생 이력에 시와 읍면동 기준으로 umd2 생성
landslide_CNT["umd2"] = landslide_CNT["sgg"] + " " + landslide_CNT["umd"]

# 날짜형 데이터
landslide_CNT["date"] = landslide_CNT["date"].astype("str")
landslide_CNT["date"] = pd.to_datetime(landslide_CNT["date"])
landslide_CNT = landslide_CNT.drop(["sd", 'umd', "sgg", "sum_cnt", "sum_hpa"], axis=1)
landslide_CNT.rename(columns={'date':'tm'}, inplace=True)
landslide_CNT["landslide"] = 1

landslide_CNT

,tm,umd2,landslide
0,2011-07-09,밀양시 내일동,1
1,2011-07-09,밀양시 단장면,1
2,2011-07-09,밀양시 무안면,1
3,2011-07-09,밀양시 부북면,1
4,2011-07-09,밀양시 산외면,1
...,...,...,...
354,2019-10-03,포항시 북구 장량동,1
355,2019-10-03,포항시 북구 중앙동,1
356,2019-10-03,포항시 북구 청하면,1
357,2019-10-03,포항시 북구 환여동,1


In [ ]:
# 날씨, 임상도, 토양도가 합쳐진 데이터와 산사태 발생 이력 데이터 병합 (umd2와 날짜 기준)

merge_final = pd.merge(merge_s_h_a, landslide_CNT, how='outer',on=['tm','umd2'])
merge_final

,tm,stnId,umd2,avgTa,minTa,maxTa,mi10MaxRn,hr1MaxRn,sumRnDur,sumRn,maxInsWs,maxWs,avgWs,minRhm,avgRhm,n99Rn,PRRCK_LARG,PRRCK_MDDL,LOCTN_ALTT,LOCTN_GRDN,EIGHT_AGL,CLZN_CD,TPGRP_TPCD,PRDN_FOM_C,SLANT_TYP,SLDPT_TPCD,SCSTX_CD,SLTP_CD,STORUNST,FROR_CD,FRTP_CD,KOFTR_GROU,DMCLS_CD,AGCLS_CD,HEIGHT,landslide
0,2011-01-01,115,울릉군 울릉읍,0.8,-1.1,2.1,0.0,0.0,19.33,28.0,29.4,19.0,11.1,65.0,82.5,5.8,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,NaN
1,2011-01-02,115,울릉군 울릉읍,0.1,-0.9,1.3,0.0,0.0,22.83,24.8,14.5,8.5,6.0,65.0,91.5,29.7,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,NaN
2,2011-01-03,115,울릉군 울릉읍,-0.3,-1.4,0.8,0.0,0.0,12.83,7.5,15.9,10.2,6.5,61.0,80.3,0.3,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,NaN
3,2011-01-04,115,울릉군 울릉읍,1.7,0.4,3.9,0.0,0.0,0.00,0.0,11.2,6.8,3.1,53.0,63.1,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,NaN
4,2011-01-05,115,울릉군 울릉읍,1.2,-1.2,2.6,0.0,0.0,7.25,1.8,13.9,6.9,4.1,42.0,61.5,12.3,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2093785,2019-12-27,295,남해군 창선면,3.3,-1.9,7.4,0.0,0.0,0.00,0.0,5.8,3.4,1.8,30.0,50.0,0.0,1,11,204.6,31.4,322.5,4,7,1,1,20,2,1,1.0,2.0,1.0,15.0,2.0,4.0,12.0,NaN
2093786,2019-12-28,295,남해군 창선면,2.2,-1.9,8.1,0.0,0.0,0.00,0.0,2.7,1.6,0.9,33.0,56.1,0.0,1,11,204.6,31.4,322.5,4,7,1,1,20,2,1,1.0,2.0,1.0,15.0,2.0,4.0,12.0,NaN
2093787,2019-12-29,295,남해군 창선면,3.2,-0.5,5.8,0.0,0.0,0.00,8.5,2.6,1.4,0.8,53.0,75.8,0.0,1,11,204.6,31.4,322.5,4,7,1,1,20,2,1,1.0,2.0,1.0,15.0,2.0,4.0,12.0,NaN
2093788,2019-12-30,295,남해군 창선면,6.8,3.6,12.6,0.0,0.0,0.00,0.5,6.1,3.4,1.2,50.0,78.9,0.0,1,11,204.6,31.4,322.5,4,7,1,1,20,2,1,1.0,2.0,1.0,15.0,2.0,4.0,12.0,NaN


In [ ]:
# 산사태가 발생하지 않은 날짜는 0으로 처리

merge_final_real = merge_final
merge_final_real["landslide"] = merge_final["landslide"].fillna(0)
merge_final_real.isnull().sum()

tm               0
stnId            0
umd2             0
avgTa         1664
minTa          159
maxTa          214
mi10MaxRn        0
hr1MaxRn         0
sumRnDur         0
sumRn            0
maxInsWs       437
maxWs          543
avgWs          911
minRhm        1515
avgRhm        3626
n99Rn            0
PRRCK_LARG       0
PRRCK_MDDL       0
LOCTN_ALTT       0
LOCTN_GRDN       0
EIGHT_AGL        0
CLZN_CD          0
TPGRP_TPCD       0
PRDN_FOM_C       0
SLANT_TYP        0
SLDPT_TPCD       0
SCSTX_CD         0
SLTP_CD          0
STORUNST         0
FROR_CD          0
FRTP_CD          0
KOFTR_GROU       0
DMCLS_CD         0
AGCLS_CD         0
HEIGHT           0
landslide        0
dtype: int64

In [ ]:
merge_final_real

,tm,stnId,umd2,avgTa,minTa,maxTa,mi10MaxRn,hr1MaxRn,sumRnDur,sumRn,maxInsWs,maxWs,avgWs,minRhm,avgRhm,n99Rn,PRRCK_LARG,PRRCK_MDDL,LOCTN_ALTT,LOCTN_GRDN,EIGHT_AGL,CLZN_CD,TPGRP_TPCD,PRDN_FOM_C,SLANT_TYP,SLDPT_TPCD,SCSTX_CD,SLTP_CD,STORUNST,FROR_CD,FRTP_CD,KOFTR_GROU,DMCLS_CD,AGCLS_CD,HEIGHT,landslide
0,2011-01-01,115,울릉군 울릉읍,0.8,-1.1,2.1,0.0,0.0,19.33,28.0,29.4,19.0,11.1,65.0,82.5,5.8,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,0.0
1,2011-01-02,115,울릉군 울릉읍,0.1,-0.9,1.3,0.0,0.0,22.83,24.8,14.5,8.5,6.0,65.0,91.5,29.7,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,0.0
2,2011-01-03,115,울릉군 울릉읍,-0.3,-1.4,0.8,0.0,0.0,12.83,7.5,15.9,10.2,6.5,61.0,80.3,0.3,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,0.0
3,2011-01-04,115,울릉군 울릉읍,1.7,0.4,3.9,0.0,0.0,0.00,0.0,11.2,6.8,3.1,53.0,63.1,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,0.0
4,2011-01-05,115,울릉군 울릉읍,1.2,-1.2,2.6,0.0,0.0,7.25,1.8,13.9,6.9,4.1,42.0,61.5,12.3,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2093785,2019-12-27,295,남해군 창선면,3.3,-1.9,7.4,0.0,0.0,0.00,0.0,5.8,3.4,1.8,30.0,50.0,0.0,1,11,204.6,31.4,322.5,4,7,1,1,20,2,1,1.0,2.0,1.0,15.0,2.0,4.0,12.0,0.0
2093786,2019-12-28,295,남해군 창선면,2.2,-1.9,8.1,0.0,0.0,0.00,0.0,2.7,1.6,0.9,33.0,56.1,0.0,1,11,204.6,31.4,322.5,4,7,1,1,20,2,1,1.0,2.0,1.0,15.0,2.0,4.0,12.0,0.0
2093787,2019-12-29,295,남해군 창선면,3.2,-0.5,5.8,0.0,0.0,0.00,8.5,2.6,1.4,0.8,53.0,75.8,0.0,1,11,204.6,31.4,322.5,4,7,1,1,20,2,1,1.0,2.0,1.0,15.0,2.0,4.0,12.0,0.0
2093788,2019-12-30,295,남해군 창선면,6.8,3.6,12.6,0.0,0.0,0.00,0.5,6.1,3.4,1.2,50.0,78.9,0.0,1,11,204.6,31.4,322.5,4,7,1,1,20,2,1,1.0,2.0,1.0,15.0,2.0,4.0,12.0,0.0


In [ ]:
# 날씨, 임상도, 토양도, 산사태 발생 이력이 병합된 최종 데이터를 merge_final_real.csv 라는 이름으로 저장

merge_final_real.to_csv("merge_final_real.csv", index=False, encoding='utf-8-sig')